In [ ]:
%matplotlib inline
# Importing necessary packages:
import re
from glob import glob


import numpy as np
import datetime
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import time
import tracemalloc
import cProfile


import scipy.integrate as integrate
import scipy.special as special
from sympy import elliptic_pi

from PolyLine import *

np.set_printoptions(precision=17)

In [ ]:
ens_files = glob("./N2000D20ac2-analyze-not_full_phic-analyze-test/N*/N*.csv")

geometry = 'cylinder'

rhist_name = '_rHists'
rhist_files = PolyLine.file_reader(ens_files,extensions=['_rHists.csv','_rEdges.csv'])
rhist_dfs = PolyLine.df_of_ensembles(rhist_files,rhist_name,geometry,dtype=int,single=False)
ens_evg_rhist = PolyLine.ensemble_avg(rhist_dfs,rhist_name,geometry)

zhist_name = '_zHists'
zhist_files = PolyLine.file_reader(ens_files,extensions=['_zHists.csv','_zEdges.csv'])
zhist_dfs = PolyLine.df_of_ensembles(zhist_files,zhist_name,geometry,dtype=int,single=False)
ens_evg_zhist = PolyLine.ensemble_avg(zhist_dfs,zhist_name,geometry)

thetahist_name = '_thetaHists'
thetahist_files = PolyLine.file_reader(ens_files,extensions=['_thetaHists.csv','_thetaEdges.csv'])
thetahist_dfs = PolyLine.df_of_ensembles(thetahist_files,thetahist_name,geometry,dtype=int,single=False)
ens_evg_thetahist = PolyLine.ensemble_avg(thetahist_dfs,thetahist_name,geometry)

rFloryhist_name = '_rFloryHists'
rFloryhist_files = PolyLine.file_reader(ens_files,extensions=['_rFloryHists.csv','_rFloryEdges.csv'])
rFloryhist_dfs = PolyLine.df_of_ensembles(rFloryhist_files,rFloryhist_name,geometry,dtype=int,single=False)
ens_evg_rFloryhist = PolyLine.ensemble_avg(rFloryhist_dfs,rFloryhist_name,geometry)

fsd_name = '_fsd_t'
fsd_files = PolyLine.file_reader(ens_files,extensions=['_fsd_t.csv'])
fsd_dfs = PolyLine.df_of_ensembles(fsd_files,fsd_name,geometry)
ens_evg_fsd = PolyLine.ensemble_avg(fsd_dfs,fsd_name,geometry)

gyr_name = '_gyr_t'
gyr_files = PolyLine.file_reader(ens_files,extensions=['_gyr_t.csv'])
gyr_dfs = PolyLine.df_of_ensembles(gyr_files,gyr_name,geometry)
ens_evg_gyr = PolyLine.ensemble_avg(gyr_dfs,gyr_name,geometry)

rFlory_name = '_rFlory_t'
rFlory_files = PolyLine.file_reader(ens_files,extensions=['_rFlory_t.csv'])
rFlory_dfs = PolyLine.df_of_ensembles(rFlory_files,rFlory_name,geometry)
ens_evg_rFlory = PolyLine.ensemble_avg(rFlory_dfs,rFlory_name,geometry)

In [ ]:
properties_name = '_properties'
properties_files = PolyLine.file_reader(ens_files,extensions=['_properties.csv'])
properties_df = PolyLine.df_of_properties(properties_files, properties_name, geometry)

In [ ]:
ens_avg_properties = PolyLine.ens_avg_properties(properties_df, properties_name, geometry)

In [ ]:
df_name = '_ens_avg_r'
direction = 'cyl_radial'
ens_avg_rho_r, ens_avg_phi_r = PolyLine.rho_phi_generator(ens_evg_rhist, ens_avg_properties, df_name, geometry, direction)

In [ ]:
df_name = '_ens_avg_z'
direction = 'cyl_z'
ens_avg_rho_z = PolyLine.rho_phi_generator(ens_evg_zhist, ens_avg_properties, df_name, geometry, direction)

In [ ]:
df_name = '_ens_avg_theta'
direction = 'cyl_thata'
ens_avg_rho_theta = PolyLine.rho_phi_generator(ens_evg_thetahist, ens_avg_properties, df_name, geometry, direction)

In [ ]:
ens_avg_phi_r.plot()
ens_avg_rho_r.plot()
ens_evg_zhist.plot()
ens_evg_thetahist.plot()

In [ ]:
fig, ax = plt.subplots(nrows=1,ncols=1,figsize=(16,9))
fontsize = 24
color = 'tab:blue'
line1, = ax.plot(ens_avg_phi_r.iloc[:,0], markersize=0.1, c=color)
#ax.legend()
ax.set_xlabel(r"Cylinder radius ($\frac{D}{2a}$)",fontsize=fontsize)
ax.set_ylabel(r"$\phi_m(r)$",fontsize=fontsize)
ax.tick_params(labelsize=24)

ax2 = ax.twinx()
color = 'tab:red'
line2, = ax2.plot(ens_avg_rho_r.iloc[:,0], rho, markersize=0.1, c=color)
ax2.set_ylabel(r"$\rho_m(r)$",fontsize=fontsize)
ax2.tick_params(labelsize=24)

lines = [line1, line2]

ax.legend(lines, [l.get_label() for l in lines],loc=1,ncol=2,fontsize=fontsize)
#fig.legend()
picname = filename.split("_rHists")[0]+"_local"
plt.savefig(picname+'.pdf',dpi=300)

In [ ]:
fpath = hist_files[24]

cylindrical_shell_integrand = lambda r: 2 * np.pi * cell_attrs.dcyl * r

# the sum of rho is not equal to the bulk number density (r=infiity) natom/cell_vol
# this arises from the way we descritize the local number desnity.
rho = PolyLine.local_number_density(histo_collections, bin_edges, cylindrical_shell_integrand)

rmon = cell_attrs.dmon / 2.0
bin_centers = np.around((bin_edges[:-1] + bin_edges[1:]) / 2.0,decimals=2)

range_of_bins = PolyLine.bin_edge_index(rmon, bin_centers, bin_edges)
volume_shares = PolyLine.vol_shares(rmon, bin_centers, bin_edges, range_of_bins, PolyLine.sphere_cylinder_intersection)

# the sum of phi is not equal to the bulk volume fraction (r=infiity) natom*vol_per_atom/cell_vol
# this arises from the way we descritize the local volume fraction and the way we relate it to the local number density.
phi = PolyLine.local_volume_fraction(bin_centers, rho, volume_shares)

fig, ax = plt.subplots(nrows=1,ncols=1,figsize=(16,9))

fontsize = 24
color = 'tab:blue'
line1, = ax.plot(bin_centers, phi, label = r"$\phi_c(R=\infty)={:4.2f}$".format(cell_attrs.vfrc_crowd), markersize=0.1, c=color)
#ax.legend()
ax.set_xlabel(r"Cylinder radius ($\frac{D}{2a}$)",fontsize=fontsize)
ax.set_ylabel(r"$\phi_m(r)$",fontsize=fontsize)
ax.tick_params(labelsize=24)

ax2 = ax.twinx()
color = 'tab:red'
line2, = ax2.plot(bin_centers, rho, label = r"$\rho_c(R=\infty)={:4.2f}$".format(cell_attrs.vfrc_crowd), markersize=0.1, c=color)
ax2.set_ylabel(r"$\rho_m(r)$",fontsize=fontsize)
ax2.tick_params(labelsize=24)

lines = [line1, line2]

ax.legend(lines, [l.get_label() for l in lines],loc=1,ncol=2,fontsize=fontsize)
#fig.legend()
picname = filename.split("_rHists")[0]+"_local"
plt.savefig(picname+'.pdf',dpi=300)

In [ ]:
import os
for direct in rhist_df.columns:
    path = os.path.join("./", direct)
    os.mkdir(path)

In [ ]:
fpath = hist_files[24]

histo_collections = np.loadtxt(fpath[0],dtype=np.int32)
bin_edges = np.around(np.loadtxt(fpath[1]),decimals=2)
cell_attrs = PolyLine.cellAttributes(fpath[0],geometry='cylinder',printname=True)
filename = cell_attrs.filename
cylindrical_shell_integrand = lambda r: 2 * np.pi * cell_attrs.dcyl * r

# the sum of rho is not equal to the bulk number density (r=infiity) natom/cell_vol
# this arises from the way we descritize the local number desnity.
rho = PolyLine.local_number_density(histo_collections, bin_edges, cylindrical_shell_integrand)

rmon = cell_attrs.dmon / 2.0
bin_centers = np.around((bin_edges[:-1] + bin_edges[1:]) / 2.0,decimals=2)

range_of_bins = PolyLine.bin_edge_index(rmon, bin_centers, bin_edges)
volume_shares = PolyLine.vol_shares(rmon, bin_centers, bin_edges, range_of_bins, PolyLine.sphere_cylinder_intersection)

# the sum of phi is not equal to the bulk volume fraction (r=infiity) natom*vol_per_atom/cell_vol
# this arises from the way we descritize the local volume fraction and the way we relate it to the local number density.
phi = PolyLine.local_volume_fraction(bin_centers, rho, volume_shares)

fig, ax = plt.subplots(nrows=1,ncols=1,figsize=(16,9))

fontsize = 24
color = 'tab:blue'
line1, = ax.plot(bin_centers, phi, label = r"$\phi_c(R=\infty)={:4.2f}$".format(cell_attrs.vfrc_crowd), markersize=0.1, c=color)
#ax.legend()
ax.set_xlabel(r"Cylinder radius ($\frac{D}{2a}$)",fontsize=fontsize)
ax.set_ylabel(r"$\phi_m(r)$",fontsize=fontsize)
ax.tick_params(labelsize=24)

ax2 = ax.twinx()
color = 'tab:red'
line2, = ax2.plot(bin_centers, rho, label = r"$\rho_c(R=\infty)={:4.2f}$".format(cell_attrs.vfrc_crowd), markersize=0.1, c=color)
ax2.set_ylabel(r"$\rho_m(r)$",fontsize=fontsize)
ax2.tick_params(labelsize=24)

lines = [line1, line2]

ax.legend(lines, [l.get_label() for l in lines],loc=1,ncol=2,fontsize=fontsize)
#fig.legend()
picname = filename.split("_rHists")[0]+"_local"
plt.savefig(picname+'.pdf',dpi=300)

In [ ]:
fig, ax = plt.subplots(nrows=1,ncols=1)
nens = 0
hists = 0
phi_c = 0.0 

for filename in ens_avg_properties.filename:
    condition = ens_avg_properties['filename']==filename
    for col in rhist_dfs.columns:
        if col.split('ens')[0] == filename.split('_ens_avg')[0]:
            #print(rhist_dfs[col])
            PolyLine.histo_plotter(ax, rhist_dfs[col].values, lambda r: 2 * ens_avg_properties[condition].dcyl / 2.0 * r, rhist_dfs.index.values, label = r"$\phi_c={}$".format(ens_avg_properties[condition].vfrc_crowd), markersize=0.1)
    #ens_evg_rhist
    
#PolyLine.histo_plotter(ax, hists, lambda r: 2 * cell_attrs.dcyl / 2.0 * r, edges, label = r"$\phi_c={}$".format(phi_c), markersize=0.1)
ax.legend()
ax.set_xlabel(r"Cylinder radius ($\frac{D}{2a}$)")
ax.set_xlabel(r"Cylinder radius ($\frac{D}{2a}$)")